In [ ]:
import json
from textblob import TextBlob
from datetime import datetime,timedelta
from collections import Counter

class userprofile:
    def __init__(self,name):
        self.name=name
        self.mood_entries=[]
        self.mood_history={}
        
    def add_mood(self,entry,mood,date):
        self.mood_entries.append({"entry":entry,"mood":mood})
        self.mood_history[date]={"entry":entry,"mood":mood}

    def analyze_mood(self,entry):
        blob=TextBlob(entry)
        sentiment=blob.sentiment.polarity
        if sentiment>0:
            return "Happy"
        elif sentiment<0:
            return "Sad"
        else:
            return "Neutral"

    def suggest_activity(self,mood):
        suggestion={"Happy":"Celebrate or share your joy with a friend!",
                "Sad":"Try some self care or have a healthy snack.",
                "Neutral":"Maybe read a book or take a calm walk."}
        return suggestion.get(mood,"Do something that makes you feel good!")
    
    
    def journal_entry(self):
        today=datetime.today().strftime("%y-%m-%d")
        entry=input("How are you feeling today?").strip().lower()
        if not entry:
            print("entry cannot be empty")
            return

        mood=self.analyze_mood(entry)
        self.add_mood(entry,mood,today)
        print(f"You're in a {mood} mood!")
        print(f"Suggested activity:{self.suggest_activity(mood)}")

    def show_mood_history(self):
        if not self.mood_history:
            print("No mood entries found.")
        else:
            print("Mood history: ")
            for date, data in self.mood_history.items():
                print(f"Date: {date} | Mood: {data['mood']} | Entry: \"{data['entry']}\"")

    def weekly_summary(self,f=None):
        one_week=datetime.today() - timedelta(days=7)
        recent_entries={date:data for date,data in self.mood_history.items()
                       if datetime.strptime(date,"%y-%m-%d")>=one_week}
        output = "\nWeekly mood summary:"
        
        if not recent_entries:
            output += "No mood entries from the last 7 days."
        else:
            moods=[data['mood']for data in recent_entries.values()]
            mood_count=Counter(moods)

            output += f"\n Weekly mood summary (from {len(recent_entries)} day(s) out of 7): "
            for mood,count in mood_count.items():
                output += f"{mood}: {count} time(s)"

        if f:
            f.write(output)
        else:
            print(output)
            
    def export_to_txt(self,filename="mood_journal.txt"):
        with open(filename,'w')as f:
            f.write(f"Mood journal for {self.name}\n")
            if not self.mood_history:
                f.write("No mood entries available.\n")
            else:
                f.write("Full mood history\n")
                for date,data in sorted(self.mood_history.items()):
                    f.write(f"{date} | Mood: {data['mood']} | Entry: {data['entry']}\n")
            self.weekly_summary(f)
                

    def save_journal(self,filename):
        data_to_save={"Name":self.name,"Mood History":self.mood_history}
        with open(filename,'w')as f:
            json.dump(data_to_save,f,indent=4)
        print("journal saved successfully. ")

    @classmethod
    def load_journal(cls,filename):
        try:
            with open(filename, 'r')as f:
                data=json.load(f)
                user=cls(data['name'])
                self.mood_history=data.get("Mood History",{})
                return user
            print("Journal loaded successsfully. ")
            
        except FileNotFoundError:
            print(f"File {filename} not found.")
            return None
        

In [ ]:
if __name__=="__main__":
    filename="my_mood_journal.json"
    user=userprofile.load_journal(filename)

    if not user:
        name=input("Enter your name: ").strip()
        user=userprofile(name)

    while True:
        print("\n Menu:")
        print("1. Add Mood Entry")
        print("2. Show Mood History")
        print("3. Weekly Mood Summary")
        print("4. Export to TXT")
        print("5. Save & Exit")
        choice=input("Choose an option (1-5): ").strip()

        if choice == "1":
            user.journal_entry()
        elif choice == "2":
            user.show_mood_history()
        elif choice == "3":
            user.weekly_summary()
        elif choice == "4":
            user.export_to_txt()
        elif choice == "5":
            user.save_journal(filename)
            print("Journal saved.👋 Goodbye!")
            break
        else:
            print("Invalid choice. Please enter a number from 1 to 5.")